In [146]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from hdfs import InsecureClient

pg_url = "jdbc:postgresql://192.168.56.31:5432/postgres"
pg_properties = {"user": "postgres", "password": "Pa$$w0rd"}

spark = SparkSession.builder\
        .config("spark.driver.extraClassPath", '/opt/spark/jars/postgresql-42.2.20.jar')\
        .appName('lesson')\
        .master("local")\
        .getOrCreate()


In [153]:
film_df = spark.read.jdbc(pg_url, 'public.film', properties=pg_properties)
film_actor_df = spark.read.jdbc(pg_url, 'public.film_actor', properties=pg_properties)
actor_df = spark.read.jdbc(pg_url, 'public.actor', properties=pg_properties)
film_category_df = spark.read.jdbc(pg_url, 'public.film_category', properties=pg_properties)
category_df = spark.read.jdbc(pg_url, 'public.category', properties=pg_properties)
rental_df = spark.read.jdbc(pg_url, 'public.rental', properties=pg_properties)
inventory_df = spark.read.jdbc(pg_url, 'public.inventory', properties=pg_properties)
payment_df = spark.read.jdbc(pg_url, 'public.payment', properties=pg_properties)
address_df = spark.read.jdbc(pg_url, 'public.address', properties=pg_properties)
customer_df = spark.read.jdbc(pg_url, 'public.customer', properties=pg_properties)
city_df = spark.read.jdbc(pg_url, 'public.city', properties=pg_properties)

In [49]:
# select COUNT(*) as COUNT_IN_CATEGORY, C."name" as CATEGORY_NAME from film f
# left join film_category fc
# on F.film_id = FC.film_id
# left join category c
# on C.category_id = FC.category_id
# group by C."name"
# order by COUNT(*) desc;

df1 = film_df.join(film_category_df\
                  , film_df.film_id == film_category_df.film_id\
                  , "left")\
             .join(category_df\
                  , category_df.category_id == film_category_df.category_id\
                  , "left")\
             .select(category_df['name'])\
             .groupBy(category_df['name'])\
             .count()\
             .orderBy(F.desc('count'))

In [73]:
# select concat(a.first_name, ' ', a.last_name) actor_name, count(r.rental_id) rental_count from rental r 
# join inventory i 
# on i.inventory_id  = r.inventory_id
# join film f 
# on i.film_id = f.film_id 
# join film_actor fa 
# on fa.film_id = f.film_id 
# join actor a 
# on a.actor_id = fa.actor_id
# group by actor_name
# order by count(r.rental_id) desc;

df2 = inventory_df.join(rental_df\
                       , inventory_df.inventory_id == rental_df.inventory_id\
                       , "left")\
                  .join(film_df\
                       , inventory_df.film_id == film_df.film_id\
                       , "left")\
                  .join(film_actor_df\
                       , film_actor_df.film_id == film_df.film_id\
                       , "left")\
                  .join(actor_df\
                       , actor_df.actor_id == film_actor_df.actor_id\
                       , "left")\
            .select(F.concat(F.col("first_name"), F.lit(" "), F.col("last_name")).alias("actor_name"))\
            .groupBy('actor_name')\
            .count()\
            .orderBy(F.desc('count'))

In [98]:
# select sum(p.amount) as sum_amount, c."name" as category_name from payment p
# join rental r
# on p.rental_id = r.rental_id
# join inventory i
# on r.inventory_id = i.inventory_id 
# join film_category fc
# on i.film_id = fc.film_id 
# join category c
# on c.category_id =fc.category_id
# group by c."name" 
# order by sum(p.amount) desc
# limit 1;

df3 = payment_df.join(rental_df\
                       , payment_df.rental_id == rental_df.rental_id\
                       , "left")\
                  .join(inventory_df\
                       , rental_df.inventory_id == inventory_df.inventory_id\
                       , "left")\
                  .join(film_category_df\
                       , inventory_df.film_id == film_category_df.film_id\
                       , "left")\
                  .join(category_df\
                       , category_df.category_id == film_category_df.category_id\
                       , "left")\
            .select(category_df['name'], payment_df['amount'])\
            .groupBy(category_df['name']).agg(F.sum('amount').alias("sum_amount"))\
            .orderBy(F.desc('name'))\
            .limit(1)

In [115]:
# select f.title from film f
# left join inventory i
# on i.film_id = f.film_id
# where i.film_id is null

df4 = film_df.join(inventory_df\
                   , inventory_df.film_id == film_df.film_id\
                   , "left")\
              .filter(inventory_df.film_id.isNull()).select(film_df['title'])

In [134]:
# select count(a.first_name), a.first_name, c."name" from actor a
# join film_actor fa 
# on fa.actor_id = a.actor_id
# join film_category fc
# on fc.film_id = fa.film_id
# join category c
# on fc.category_id = c.category_id
# where c."name" = 'Children'
# group by c."name", a.first_name
# order by count(a.first_name) desc
# fetch next 3 rows with ties;

df5 = actor_df.join(film_actor_df\
                   , film_actor_df.actor_id == actor_df.actor_id\
                   , "left")\
               .join(film_category_df\
                    , film_category_df.film_id == film_actor_df.film_id\
                    , "left")\
               .join(category_df\
                    , film_category_df.category_id == category_df.category_id\
                    , "left")\
               .where(F.col('name') == 'Children')\
               .groupBy(category_df['name'], actor_df['first_name'])\
               .count()\
               .orderBy(F.desc('count'))
df5_col = df5.collect()

In [142]:
value = 0
count = 0
for el in df5_col:
    value_next = el['count']
    if (count <=2 ):
        print(str(value_next) + " " + el['first_name'] + " " + el['name'])
        value = value_next
        count += 1
    elif (count > 2 and value == value_next):
        print(str(value_next) + " " + el['first_name'] + " " + el['name'])
        value = value_next
    else:
        break
    

9 CUBA Children
7 KEVIN Children
7 RENEE Children
7 MARY Children
7 HELEN Children


In [ ]:
# select c.city,
# (select count(*) from customer ctmr
# join address a
# on a.address_id = ctmr.address_id
# where a.city_id = c.city_id
# and ctmr.active = 1) as count_of_active,
# (select count(*) from customer ctmr
# join address a
# on a.address_id = ctmr.address_id
# where a.city_id = c.city_id
# and ctmr.active = 0) as count_of_unactive
# from city c
# order by count_of_unactive desc;
  
    df5_active = address_df.join(customer_df
                               , address_df.address_id == customer_df.address_id
                               , "left")
                             .join(city_df
                               , address_df.city_id == city_df.city_id
                               , "left")
                             .select(city_df['city_id'], customer_df['active']).where(F.col('active') == 1)
                             .groupBy(city_df['city_id']).count()
    
    df5_unactive = address_df.join(customer_df
                               , address_df.address_id == customer_df.address_id
                               , "left")
                             .join(city_df
                               , address_df.city_id == city_df.city_id
                               , "left")
                             .select(city_df['city_id'], customer_df['active']).where(F.col('active') == 0)
                             .groupBy(city_df['city_id']).count()
    
    df5 = city_df.join(df5_active
                        , city_df.city_id == df5_active.city_id
                        , "left" )
                 .join(df5_unactive
                        , city_df.city_id == df5_active.city_id
                        , "left")
                 .select(city_df['city'], F.col('count'))
                 .orderBy(F.desc('count'))                    